In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import cv2
import math
import numpy as np
from skimage import measure
from skimage.measure import regionprops
from skimage.morphology import opening, erosion, dilation, disk
import matplotlib.pyplot as plt

In [0]:
TRAIN = ['232', '239', '230', '242']
TEST  = [ '263','221', '229', '246', '252']

In [0]:
TRAIN_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/PR/Laddernet/13-05-20/Train/"
TEST_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/PR/Laddernet/13-05-20/Test/"
MASK_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/4 Class PR/Masks/"

In [0]:
def updateConfusionMatrix(ConfusionMatrix, PredictedImage, GroundTruth, PatientId, axis ):
    # Opening operation
    selem = disk(20)
    #PredictedImage = erosion(PredictedImage, selem)
    GroundTruth = dilation(GroundTruth,selem)
    GroundTruthLabel = measure.label(GroundTruth)

    rows,cols = PredictedImage.shape

    #  0 is background
    for label in range(1, GroundTruthLabel.max()+1):
      masklabel = np.zeros([rows,cols])
      masklabel[GroundTruthLabel==label] = 1

      resultantimage = cv2.multiply(PredictedImage, masklabel)

      areaGT = np.sum(masklabel)
      areaSM = np.sum(resultantimage)

      if(areaSM > 0 ):
        ConfusionMatrix[axis][0] = ConfusionMatrix[axis][0] + 1       #TP
        PredictedImage[GroundTruthLabel==label] = 0
      else:
        ConfusionMatrix[axis][2] = ConfusionMatrix[axis][2] + 1       #FN
    

    predictedprops = regionprops(measure.label(PredictedImage))

    for i in range(len(predictedprops)):
      if(predictedprops[i].area >= 2500):
        ConfusionMatrix[axis][1] += 1                                #FP

In [0]:
def getPatientWiseCount(ConfusionMatrix, PredictedPatient, GroundTruthPatient, PatientId):

  for imagenum in range(PredictedPatient.shape[0]):
  
    PredictedImage = PredictedPatient[imagenum]
    GroundTruth    = GroundTruthPatient[imagenum]
    rows,cols = PredictedImage.shape

    # dividing the image into strong,weak,intermediate
    predictedStrong = np.zeros([rows,cols])
    predictedWeak   = np.zeros([rows,cols])
    predictedIntermediate = np.zeros([rows,cols])

    gtStrong = np.zeros([rows,cols])
    gtWeak = np.zeros([rows,cols])
    gtIntermediate = np.zeros([rows,cols])

    predictedStrong[PredictedImage==255] = 1
    predictedIntermediate [PredictedImage==170] = 1
    predictedWeak[PredictedImage==85] = 1

    gtStrong[GroundTruth==255] = 1
    gtIntermediate[GroundTruth==170] = 1
    gtWeak[GroundTruth==85] = 1

    # 0th axis in confusion matrix is strong 
    # 1st axis in confusion matrix is intermediate 
    # 2nd axis in confusion matrix is weak
    updateConfusionMatrix(ConfusionMatrix, predictedStrong, gtStrong, PatientId, 0)
    updateConfusionMatrix(ConfusionMatrix, predictedIntermediate, gtIntermediate, PatientId, 1)
    updateConfusionMatrix(ConfusionMatrix, predictedWeak, gtWeak, PatientId, 2)

**SM = Segmented Mask**

**GT = Ground Truth** 

In [7]:
train_confusion_matrix = np.zeros([ 3, 3])
test_confusion_matrix  = np.zeros([ 3, 3])

for patientid in range(len(TRAIN)):
  patientSM = np.load(TRAIN_PATH + TRAIN[patientid] + '.npy')
  patientGT      = np.load(MASK_PATH + 'IHC ' + TRAIN[patientid] + '.npy')
  getPatientWiseCount(train_confusion_matrix, patientSM, patientGT, patientid)
  print(TRAIN[patientid] + ' done.')
  print(train_confusion_matrix)

232 done.
[[10.  0.  9.]
 [20. 17.  2.]
 [36.  8. 13.]]
239 done.
[[312.   8.  19.]
 [222.  68.  16.]
 [141.  20.  26.]]
230 done.
[[312.   8.  19.]
 [266.  68.  26.]
 [141.  21.  26.]]
242 done.
[[313.   8.  19.]
 [266.  68.  27.]
 [141.  21.  26.]]


In [8]:
for patientid in range(len(TEST)):
  patientSM = np.load(TEST_PATH + TEST[patientid] + '.npy')
  patientGT      = np.load(MASK_PATH + 'IHC ' + TEST[patientid] + '.npy')
  getPatientWiseCount(test_confusion_matrix, patientSM, patientGT, patientid)
  print(TEST[patientid] + ' done.')
  print(test_confusion_matrix)

263 done.
[[106.  19.  20.]
 [155. 155.  26.]
 [130.   8.  38.]]
221 done.
[[108.  19.  21.]
 [155. 155.  26.]
 [131.  11.  38.]]
229 done.
[[109.  19.  21.]
 [155. 155.  26.]
 [131.  11.  40.]]
246 done.
[[109.  19.  21.]
 [155. 156.  26.]
 [131.  14.  40.]]
252 done.
[[109.  19.  21.]
 [155. 156.  26.]
 [131.  14.  40.]]


In [0]:
def getMetrics(ConfusionMatrix, axis):
  precision = ConfusionMatrix[axis][0] / (ConfusionMatrix[axis][0] + ConfusionMatrix[axis][1] )
  recall    = ConfusionMatrix[axis][0] / (ConfusionMatrix[axis][0] + ConfusionMatrix[axis][2] )
  dice      = (2 * precision * recall) / (precision + recall)
  return precision, recall, dice

In [10]:
strain_precision, strain_recall, strain_dice = getMetrics(train_confusion_matrix, 0)
itrain_precision, itrain_recall, itrain_dice = getMetrics(train_confusion_matrix, 1)
wtrain_precision, wtrain_recall, wtrain_dice = getMetrics(train_confusion_matrix, 2)

print('TRAIN :\n\n')
print("CLASS\t\tPRECISION\t\tRECALL\t\t\tDICE")
print('Strong\t\t' + str(strain_precision) + '\t' + str(strain_recall) + '\t' + str(strain_dice))
print('Intermediate\t' + str(itrain_precision) +  '\t' + str(itrain_recall) + '\t' + str(itrain_dice))
print('Weak\t\t' + str(wtrain_precision) +   '\t' + str(wtrain_recall) + '\t' + str(wtrain_dice))

TRAIN :


CLASS		PRECISION		RECALL			DICE
Strong		0.9750778816199377	0.9427710843373494	0.9586523736600306
Intermediate	0.7964071856287425	0.9078498293515358	0.8484848484848484
Weak		0.8703703703703703	0.844311377245509	0.8571428571428571


In [11]:
stest_precision, stest_recall, stest_dice = getMetrics(test_confusion_matrix, 0)
itest_precision, itest_recall, itest_dice = getMetrics(test_confusion_matrix, 1)
wtest_precision, wtest_recall, wtest_dice = getMetrics(test_confusion_matrix, 2)

print('TEST :\n\n')
print("CLASS\t\tPRECISION\t\tRECALL\t\t\tDICE")
print('Strong\t\t' + str(stest_precision) + '\t' + str(stest_recall) + '\t' + str(stest_dice))
print('Intermediate\t' + str(itest_precision) +  '\t' + str(itest_recall) + '\t' + str(itest_dice))
print('Weak\t\t' + str(wtest_precision) +   '\t' + str(wtest_recall) + '\t' + str(wtest_dice))

TEST :


CLASS		PRECISION		RECALL			DICE
Strong		0.8515625	0.8384615384615385	0.8449612403100775
Intermediate	0.4983922829581994	0.856353591160221	0.6300813008130082
Weak		0.903448275862069	0.7660818713450293	0.8291139240506329
